In [16]:
#cmost_config

import numpy as np

class CMOSTConfig:
    def __init__(self):
        # ==========================================
        # 1. Risk Settings (from Risk_Settings.m)
        # ==========================================
        # 환자 개인별 위험도 분포를 만들기 위한 백분위수 (예시값)
        # MATLAB의 handles.Variables.Ind_Risk_Percentiles 에 해당
        self.risk_percentiles = [0.1, 0.2, 0.3, 0.5, 1.0, 1.5, 2.0, 3.0, 5.0, 8.0, 10.0, 15.0]
        
        # 보간법을 통해 500개 구간의 위험도 분포 생성 (AdjustRiskGraph 함수 이식)
        self.individual_risk_dist = self._interpolate_risk(self.risk_percentiles)

        # 자연사 파라미터 (NumberCrunching.m의 AgeProgression 등 추상화)
        self.prob_onset_base = 0.04      # 기본 용종 발생률
        self.prob_growth = 0.15          # 용종 성장 확률 (Small -> Large)
        self.prob_malignancy = 0.05      # 암화 확률 (Large -> Cancer)
        self.prob_cancer_progress = 0.4  # 암 기수 진행 확률

        # ==========================================
        # 2. Screening Settings (from Screening_Settings.m)
        # ==========================================
        # 민감도(Sensitivity): [P1, P2, P3, P4, P5, P6, Ca1, Ca2, Ca3, Ca4]
        # P1~P6: 용종 크기별 (1mm ~ >10mm), Ca1~Ca4: 암 병기별
        
        self.sensitivities = {
            # 대장내시경 (Colonoscopy): 용종 발견율이 높음 + 치료 가능
            'colonoscopy': np.array([0.70, 0.80, 0.90, 0.95, 0.98, 0.99, 0.95, 0.98, 0.99, 0.99]),
            
            # 분변잠혈검사 (FOBT): 용종 발견율 낮음, 암 발견율은 중간
            'fobt':        np.array([0.05, 0.05, 0.10, 0.15, 0.20, 0.25, 0.50, 0.60, 0.70, 0.80]),
            
            # Septin9 등 다른 검사 추가 가능
            'sept9':       np.array([0.01, 0.02, 0.05, 0.10, 0.15, 0.20, 0.60, 0.75, 0.85, 0.90])
        }

        # 비용 및 보상 설정 (RL 학습용)
        self.cost = {
            'wait': 0,
            'fobt': 0.1,        # 저렴
            'colonoscopy': 0.5  # 비쌈 (기준점)
        }
        self.reward = {
            'alive': 0.1,       # 생존 보상
            'polyp_removed': 5.0, # 예방 보상
            'cancer_detected': 100.0, # 조기 발견 보상
            'death_penalty': -5000.0 # 사망 페널티
        }

    def _interpolate_risk(self, percentiles):
        """
        Risk_Settings.m 의 'AdjustRiskGraph' 함수 로직 구현
        주어진 백분위수 점들을 선형 보간하여 전체 인구의 위험도 분포를 생성
        """
        # x축 기준점 (MATLAB 코드의 Values 배열)
        x_points = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 97, 100]) * 5
        
        # 0~500 범위의 분포 생성
        risk_dist = np.zeros(501)
        risk_dist[:x_points[0]] = percentiles[0]
        
        prev_x = x_points[0]
        prev_y = percentiles[0]
        
        for i in range(1, len(x_points)):
            curr_x = x_points[i]
            curr_y = percentiles[i]
            
            # 선형 보간
            for x in range(prev_x, curr_x):
                risk_dist[x] = prev_y + (curr_y - prev_y) * (x - prev_x) / (curr_x - prev_x)
            
            prev_x = curr_x
            prev_y = curr_y
            
        return risk_dist

In [17]:
#based on NumberCrunching_1000.m
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from cmost_config import CMOSTConfig

class CmostEnv(gym.Env):
    """
    CMOST 기반 대장암 스크리닝 강화학습 환경
    Reference: NumberCrunching_10000.m
    """
    def __init__(self):
        super(CmostEnv, self).__init__()
        self.cfg = CMOSTConfig()
        
        # --- Action Space ---
        # 0: 대기 (Wait)
        # 1: 분변잠혈검사 (FOBT)
        # 2: 대장내시경 (Colonoscopy)
        self.action_space = spaces.Discrete(3)
        
        # --- Observation Space ---
        # [0]: 현재 나이 (normalized 0~1)
        # [1]: 증상 유무 (0 or 1) - Symptom develops logic
        # [2]: 마지막 검사 종류 (0:없음, 1:FOBT, 2:Colo)
        # [3]: 마지막 검사 결과 (0:정상, 1:용종, 2:암)
        # [4]: 마지막 검사로부터 경과 시간 (normalized)
        self.observation_space = spaces.Box(low=0, high=1, shape=(5,), dtype=np.float32)
        
        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # 1. 환자 초기화 (NumberCrunching.m: INITIALIZE 부분)
        self.age = 50 # 시작 나이
        self.max_age = 90
        
        # 환자 고유 위험도 할당 (Risk_Settings.m: IndividualRisk 적용)
        self.individual_risk = np.random.choice(self.cfg.individual_risk_dist)
        
        # Hidden States (Preclinical - Agent는 볼 수 없음)
        # 용종 리스트: 각 요소는 용종의 크기 단계 (1~6)
        self.hidden_polyps = [] 
        # 암 상태: None 또는 병기 (1~4)
        self.hidden_cancer = None 
        # 증상 타이머 (암 발생 시 설정됨)
        self.symptom_timer = -1 
        
        # Observation History
        self.last_test_type = 0
        self.last_test_result = 0
        self.time_since_last_test = 0
        
        return self._get_obs(), {}

    def step(self, action):
        reward = 0
        done = False
        info = {}
        
        # ====================================================
        # 1. Action 수행 (Screening_Settings.m & Colonoscopy 함수)
        # ====================================================
        test_performed = False
        
        if action == 1: # FOBT
            reward -= self.cfg.cost['fobt']
            test_performed = True
            detected, finding_type = self._perform_screening('fobt')
            
            if detected:
                self.last_test_result = finding_type # 1:용종, 2:암
                # FOBT 양성이면 보통 내시경을 하러 가므로(치료 기회) 약간의 보상
                reward += 0.5 
            else:
                self.last_test_result = 0

        elif action == 2: # Colonoscopy
            reward -= self.cfg.cost['colonoscopy']
            test_performed = True
            detected, finding_type = self._perform_screening('colonoscopy', remove_polyp=True)
            
            if detected:
                self.last_test_result = finding_type
                if finding_type == 1: # 용종 발견 및 제거
                    reward += self.cfg.reward['polyp_removed']
                elif finding_type == 2: # 암 발견
                    reward += self.cfg.reward['cancer_detected']
                    # 암 발견 시 시뮬레이션 종료 (치료 모드로 전환 가정)
                    done = True 
            else:
                self.last_test_result = 0

        if test_performed:
            self.last_test_type = action
            self.time_since_last_test = 0
        else:
            self.time_since_last_test += 1

        # ====================================================
        # 2. 자연사 진행 (NumberCrunching.m: Natural History)
        # ====================================================
        self._update_natural_history()

        # ====================================================
        # 3. 종료 조건 (Death / Max Age)
        # ====================================================
        self.age += 1
        reward += self.cfg.reward['alive'] # 1년 생존 보상
        
        # (1) 암으로 인한 사망 체크 (증상 발현 후 방치된 경우)
        if self.hidden_cancer and self.symptom_timer == 0:
            # 증상이 발현되었는데(Symptom develops), 치료되지 않음 -> 사망 위험
            if np.random.rand() < 0.5: # 간단화된 사망 확률
                done = True
                reward += self.cfg.reward['death_penalty']
                info['cause_of_death'] = 'cancer'

        # (2) 자연사 (LifeTable 적용 가능하나 여기선 90세로 단순화)
        if self.age >= self.max_age:
            done = True
            info['cause_of_death'] = 'natural'

        return self._get_obs(), reward, done, False, info

    def _update_natural_history(self):
        """ NumberCrunching.m 의 핵심 확률 로직 이식 """
        
        # A. 용종 발생 (New Polyp)
        # MATLAB: if rand < PolypRate(z) ...
        current_onset_prob = self.cfg.prob_onset_base * self.individual_risk
        if np.random.rand() < current_onset_prob:
            # 크기 1단계 용종 생성
            self.hidden_polyps.append(1) 

        # B. 용종 성장 (Progression)
        # MATLAB: AgeProgression * LocationProgression ...
        for i in range(len(self.hidden_polyps)):
            # 간단화: 일정 확률로 크기 +1
            if np.random.rand() < self.cfg.prob_growth:
                self.hidden_polyps[i] += 1
        
        # C. 암화 (Malignancy)
        # MATLAB: if Polyp.Polyps(z, f) > 6 ... Ca.Cancer(z, l2) = 7
        # 용종이 6단계를 넘어가면 암으로 변함
        for i in reversed(range(len(self.hidden_polyps))):
            if self.hidden_polyps[i] > 6:
                if self.hidden_cancer is None:
                    self.hidden_cancer = 1 # 1기 암 발생
                    self.symptom_timer = 5 # 5년 뒤 증상 발현 (Sojourn time)
                self.hidden_polyps.pop(i) # 용종은 암이 되었으므로 리스트에서 제거

        # D. 암 진행 (Cancer Progression)
        if self.hidden_cancer is not None:
            if self.hidden_cancer < 4: # 4기까지 진행
                if np.random.rand() < self.cfg.prob_cancer_progress:
                    self.hidden_cancer += 1
            
            # 증상 타이머 감소
            if self.symptom_timer > 0:
                self.symptom_timer -= 1

    def _perform_screening(self, test_type, remove_polyp=False):
        """ Screening_Settings.m 의 민감도 적용 """
        sens_table = self.cfg.sensitivities[test_type]
        detected = False
        finding_type = 0 # 0:없음, 1:용종, 2:암
        
        # 1. 암 탐지 시도
        if self.hidden_cancer is not None:
            # 암 병기(1~4)에 따른 민감도 인덱스: 6~9 (Python index 6,7,8,9)
            sens_idx = 5 + self.hidden_cancer 
            sensitivity = sens_table[sens_idx]
            if np.random.rand() < sensitivity:
                detected = True
                finding_type = 2
                return detected, finding_type # 암 찾으면 즉시 리턴

        # 2. 용종 탐지 시도
        # 발견된 용종들을 저장할 임시 리스트
        detected_polyps_indices = []
        
        for i, size in enumerate(self.hidden_polyps):
            # 용종 크기(1~6)에 따른 민감도 인덱스: 0~5
            sens_idx = min(size - 1, 5)
            sensitivity = sens_table[sens_idx]
            
            if np.random.rand() < sensitivity:
                detected = True
                finding_type = 1
                detected_polyps_indices.append(i)
        
        # 대장내시경인 경우 발견된 용종 제거 (Polypectomy)
        if remove_polyp and detected_polyps_indices:
            # 인덱스가 꼬이지 않게 뒤에서부터 제거
            for i in sorted(detected_polyps_indices, reverse=True):
                self.hidden_polyps.pop(i)
                
        return detected, finding_type

    def _get_obs(self):
        # Normalize age
        norm_age = (self.age - 50) / (90 - 50)
        
        # 증상 유무 (암이 있고 타이머가 0이면 증상 있음)
        has_symptom = 1.0 if (self.hidden_cancer and self.symptom_timer <= 0) else 0.0
        
        # 경과 시간 Normalize (최대 10년으로 가정)
        norm_time = min(self.time_since_last_test, 10) / 10.0
        
        return np.array([
            norm_age,
            has_symptom,
            self.last_test_type / 2.0,    # 0, 0.5, 1.0
            self.last_test_result / 2.0,  # 0, 0.5, 1.0
            norm_time
        ], dtype=np.float32)

In [18]:
from cmost_env import CmostEnv

In [19]:
env = CmostEnv()
obs, _ = env.reset()

In [20]:
print("초기 상태:", obs)

초기 상태: [0. 0. 0. 0. 0.]


In [21]:
for _ in range(10):
    # 랜덤 행동 (0:대기, 1:FOBT, 2:내시경)
    action = env.action_space.sample() 
    obs, reward, done, _, info = env.step(action)
    
    print(f"행동: {action}, 보상: {reward:.1f}, 종료: {done}")
    if done:
        print("에피소드 종료:", info)
        break

행동: 1, 보상: 0.0, 종료: False
행동: 1, 보상: 0.0, 종료: False
행동: 2, 보상: -0.9, 종료: False
행동: 1, 보상: 0.0, 종료: False
행동: 1, 보상: 0.0, 종료: False
행동: 2, 보상: 0.1, 종료: False
행동: 2, 보상: 0.1, 종료: False
행동: 0, 보상: 0.1, 종료: False
행동: 2, 보상: -0.9, 종료: False
행동: 1, 보상: 0.0, 종료: False


In [22]:
# pip install stable-baselines3 shimmy

In [ ]:
#RL training script
#vectorization of env -> PPO model creation -> Learn
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback

# --- [0] 이전에 만든 Config와 Env 클래스가 위에 정의되어 있다고 가정합니다 ---
# (만약 정의 안 되어 있다면, 이전 답변의 코드를 먼저 실행해주세요)

# --- [1] 환경 생성 및 래핑 ---
# DummyVecEnv는 여러 환경을 동시에 돌릴 수 있게 해주는데, 
# 여기서는 1개만 돌리지만 호환성을 위해 사용합니다.
env = DummyVecEnv([lambda: CmostEnv()])

# --- [2] DRL 모델 정의 (PPO 알고리즘) ---
# MlpPolicy: 입력이 이미지(CNN)가 아니라 수치 벡터(Vector)일 때 사용하는 신경망 구조
# verbose=1: 학습 과정을 로그로 출력
model = PPO(
    "MlpPolicy", 
    env, 
    verbose=1, 
    learning_rate=0.0003,    # 학습률 (너무 크면 발산, 너무 작으면 느림)
    gamma=0.999,              # 할인율 (미래 보상을 얼마나 중요하게 여길지)
    ent_coef=0.01            # 탐험 계수 (새로운 행동을 시도해보는 비율)
)

print("🚀 학습 시작! (잠시만 기다려주세요...)")

# --- [3] 모델 학습 (Training) ---
# total_timesteps: 에이전트가 환경과 상호작용하는 총 횟수 (단위: 년)
# 10만 번 정도면 간단한 정책은 배울 수 있습니다. (성능 높이려면 100만 이상 추천)
model.learn(total_timesteps=500000)

print("✅ 학습 완료!")

# --- [4] 모델 저장 ---
model.save("cmost_ppo_agent")
print("💾 모델이 'cmost_ppo_agent.zip'으로 저장되었습니다.")


Using cuda device
🚀 학습 시작! (잠시만 기다려주세요...)
-----------------------------
| time/              |      |
|    fps             | 1271 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1053       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.02574918 |
|    clip_fraction        | 0.285      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.08      |
|    explained_variance   | -0.00957   |
|    learning_rate        | 0.0003     |
|    loss                 | 1.29       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0371    |
|    value_loss           | 3.3        |
----------------------------------------
---------

In [ ]:
#evaluation (vs random)

# --- [5] 학습된 모델 성능 테스트 ---

def run_simulation(agent_model, episodes=10):
    """ 에이전트를 이용해 환자 시뮬레이션을 돌리고 결과를 출력하는 함수 """
    # 테스트용 환경 별도 생성
    test_env = CmostEnv()
    total_rewards = []
    lifespans = []
    deaths_by_cancer = 0
    
    print(f"\n--- 시뮬레이션 시작 ({episodes}명 환자) ---")
    
    for ep in range(episodes):
        obs, _ = test_env.reset()
        done = False
        ep_reward = 0
        actions = []
        
        while not done:
            # 모델이 있으면 예측값 사용, 없으면 랜덤 행동
            if agent_model:
                action, _ = agent_model.predict(obs, deterministic=True)
            else:
                action = test_env.action_space.sample()
                
            obs, reward, done, _, info = test_env.step(action)
            ep_reward += reward
            actions.append(action)
            
        total_rewards.append(ep_reward)
        lifespans.append(test_env.age)
        if info.get('cause_of_death') == 'cancer':
            deaths_by_cancer += 1
            
        # 처음 3명만 로그 자세히 출력
        if ep < 3:
            print(f"환자 {ep+1}: 사망나이 {test_env.age}세, 사인: {info.get('cause_of_death')}")
            # 행동 통계 (0:대기, 1:FOBT, 2:내시경)
            unique, counts = np.unique(actions, return_counts=True)
            action_dict = dict(zip(unique, counts))
            print(f"   -> 검사 횟수: 내시경 {action_dict.get(2, 0)}회, FOBT {action_dict.get(1, 0)}회")
            print(f"   -> 총 보상: {ep_reward:.2f}")

    print("-" * 30)
    print(f"평균 수명: {np.mean(lifespans):.1f}세")
    print(f"평균 보상: {np.mean(total_rewards):.2f}")
    print(f"암 사망률: {(deaths_by_cancer/episodes)*100:.1f}%")

# 1. 학습 전 (랜덤) 성능
print("\n[비교 1] 학습 전 (Random Agent)")
run_simulation(None, episodes=100)

# 2. 학습 후 (PPO) 성능
print("\n[비교 2] 학습 후 (PPO Agent)")
run_simulation(model, episodes=100)


[비교 1] 학습 전 (Random Agent)

--- 시뮬레이션 시작 (100명 환자) ---
환자 1: 사망나이 90세, 사인: natural
   -> 검사 횟수: 내시경 13회, FOBT 8회
   -> 총 보상: -6.80
환자 2: 사망나이 90세, 사인: natural
   -> 검사 횟수: 내시경 14회, FOBT 16회
   -> 총 보상: -8.60
환자 3: 사망나이 90세, 사인: natural
   -> 검사 횟수: 내시경 10회, FOBT 13회
   -> 총 보상: -7.30
------------------------------
평균 수명: 90.0세
평균 보상: -8.19
암 사망률: 0.0%

[비교 2] 학습 후 (PPO Agent)

--- 시뮬레이션 시작 (100명 환자) ---
환자 1: 사망나이 90세, 사인: natural
   -> 검사 횟수: 내시경 0회, FOBT 13회
   -> 총 보상: 4.20
환자 2: 사망나이 90세, 사인: natural
   -> 검사 횟수: 내시경 0회, FOBT 13회
   -> 총 보상: 4.70
환자 3: 사망나이 90세, 사인: natural
   -> 검사 횟수: 내시경 0회, FOBT 13회
   -> 총 보상: 4.70
------------------------------
평균 수명: 88.1세
평균 보상: -1.47
암 사망률: 6.0%


##실제 수치 가지고오기 from Default_Benchmark.m

In [ ]:
#.mat 파일에서 수치 가져오기. 

import scipy.io
import numpy as np

# 1. 설정 파일 로드 (CMOST13.mat이 기본 설정일 가능성이 높음)
# 경로를 실제 파일 위치에 맞게 수정하세요.
mat_data = scipy.io.loadmat('Settings/CMOST13.mat')

# 2. 데이터 구조 확인 (Key 값들을 출력해봅니다)
print("저장된 변수 목록:", mat_data.keys())

# 3. 실제 값 추출 예시
# 'Variables'라는 키 안에 데이터가 뭉쳐 있을 가능성이 큽니다.
if 'Variables' in mat_data:
    variables = mat_data['Variables'][0, 0] # 구조체 접근
    
    # 구조체 내부 필드 이름 확인
    print("\nVariables 내부 필드:", variables.dtype.names)
    
    # 예: Screening 설정값 접근 (필드명이 'Screening'이라고 가정)
    # 실제 필드명을 보고 아래 코드를 수정해야 합니다.
    # print(variables['Screening'])

저장된 변수 목록: dict_keys(['__header__', '__version__', '__globals__', 'temp'])


In [ ]:
import scipy.io
import numpy as np

# 1. 파일 다시 로드 (경로는 본인 환경에 맞게 수정)
mat_data = scipy.io.loadmat('cmost/Settings/CMOST13.mat')

# 2. 'temp' 꺼내기
temp_data = mat_data['temp']

# 3. 내부 필드(변수명) 확인하기
# MATLAB 구조체는 numpy structured array로 불러와집니다.
# dtype.names를 찍으면 안에 'Screening', 'Risk' 같은 이름이 있는지 볼 수 있습니다.
print("=== [temp] 내부의 변수 목록 ===")
print(temp_data.dtype.names)

# -----------------------------------------------------------
# 예상 시나리오: 'Variables'라는 이름이 나오거나, 
# 바로 'Screening', 'Risk' 등이 나올 수 있습니다.
# -----------------------------------------------------------

# [Case A] 만약 출력 결과에 'Variables'가 있다면:
if 'Variables' in temp_data.dtype.names:
    variables = temp_data['Variables'][0, 0]
    print("\n=== [Variables] 내부 필드 ===")
    print(variables.dtype.names)
    
    # 여기서 'Screening'이나 'Risk'를 찾아서 값을 봅니다.
    # 예: 내시경 민감도 확인
    # print(variables['Screening'][0, 0]['Colonoscopy']) 

# [Case B] 만약 바로 'Screening' 등이 있다면:
else:
    print("\n바로 데이터 접근 가능")
    # print(temp_data['Screening'])

FileNotFoundError: [Errno 2] No such file or directory: 'cmost/Settings/CMOST13.mat'

In [ ]:
import scipy.io
import numpy as np
import sys

# 출력 제한 해제 (중간 생략 방지)
np.set_printoptions(threshold=sys.maxsize)

def inspect_cmost_mat(file_path):
    print(f"📂 파일 로딩 중: {file_path}")
    print("=" * 120)
    
    try:
        # 1. 파일 로드
        mat_data = scipy.io.loadmat(file_path)
        
        # 2. 'temp' 구조체 접근
        if 'temp' not in mat_data:
            print("⚠️ 'temp' 변수를 찾을 수 없습니다.")
            return

        # MATLAB struct는 [0,0]으로 접근해야 함
        temp_struct = mat_data['temp'][0, 0]
        field_names = temp_struct.dtype.names
        
        # 헤더 출력
        print(f"{'No.':<4} | {'변수명 (Name)':<35} | {'크기 (Shape)':<12} | {'실제 값 미리보기 (Preview)'}")
        print("-" * 120)
        
        # 3. 모든 필드 순회하며 출력
        for i, name in enumerate(field_names):
            val = temp_struct[name]
            
            # (1) 크기(Shape) 처리
            shape_str = str(val.shape) if hasattr(val, 'shape') else "Scalar"
            
            # (2) 실제 값(Value) 미리보기 처리
            val_flat = np.array(val).flatten() # 1차원으로 펴서 보기 좋게 만듦
            
            # 문자열인 경우
            if val.dtype.kind in {'U', 'S'}:
                # MATLAB 문자열은 보통 배열 안에 갇혀 있어서 꺼내줌
                preview = str(val[0]) if val.size > 0 else ""
            
            # 숫자인 경우
            else:
                if val_flat.size == 0:
                    preview = "[] (Empty)"
                elif val_flat.size == 1:
                    preview = str(val_flat[0]) # 스칼라 값 그대로 출력
                elif val_flat.size <= 5:
                    # 데이터가 5개 이하면 전체 출력
                    preview = str(val_flat)
                else:
                    # 데이터가 많으면 앞 3개, 뒤 2개만 보여줌
                    preview = f"[{val_flat[0]:.4f}, {val_flat[1]:.4f}, {val_flat[2]:.4f} ... {val_flat[-2]:.4f}, {val_flat[-1]:.4f}]"
            
            # 한 줄 출력
            print(f"{i+1:03d}  | {name:<35} | {shape_str:<12} | {preview}")

        print("=" * 120)
        print(f"✅ 총 {len(field_names)}개의 변수 확인 완료.")

    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요: {file_path}")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# --- 실행 ---
file_path = 'Settings/CMOST13.mat'
inspect_cmost_mat(file_path)

📂 파일 로딩 중: Settings/CMOST13.mat
No.  | 변수명 (Name)                          | 크기 (Shape)   | 실제 값 미리보기 (Preview)
------------------------------------------------------------------------------------------------------------------------
001  | CurrentPath                         | (1,)         | /Users/misselwb/Documents/CMOST_Data/Data/Calibrating_CMOST13_01072016/
002  | ResultsPath                         | (1,)         | /cluster/home/misselwb/Data_Calib//Calibrating_CMOST13_01072016
003  | Settings_Name                       | (1,)         | CMOST13_02072016
004  | Comment                             | (1,)         | no comment please
005  | Identification                      | (1,)         | This_is_a_genuine_PolypCalculator_File
006  | LifeTable                           | (151, 2)     | [0.0076, 0.0061, 0.0005 ... 0.3124, 0.2742]
007  | NewPolypRate                        | (1, 20)      | [0.0000, 0.0001, 0.0001 ... 0.0229, 0.0234]
008  | EarlyProgressionRate                | (1, 

###필요한 변수들 실제 수치값 반영한 env --> DRL

In [ ]:
#config
import numpy as np
import scipy.io

class CMOSTConfig:
    def __init__(self, mat_file_path='Settings/CMOST13.mat'):
        print(f"Loading Optimized Configuration from {mat_file_path}...")
        
        try:
            mat_data = scipy.io.loadmat(mat_file_path)
            self.data = mat_data['temp'][0, 0]
            print("✅ 데이터 로드 성공: 핵심 변수 추출 및 보간 시작")
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            self.data = None

        if self.data is not None:
            # =========================================================
            # [1] 인구학적 통계 (Demographics)
            # =========================================================
            # 여성 비율 (성별에 따른 발병률 차이 적용을 위해 필수)
            self.frac_female = self._get_scalar('fraction_female')

            # =========================================================
            # [2] 생물학적 파라미터 (Natural History) - 성별 구분
            # =========================================================
            # 남성 데이터 (기본값) -> 100세까지 보간(Interpolation)
            self.male_onset = self._interpolate(self.data['NewPolypRate'])
            self.male_early = self._interpolate(self.data['EarlyProgressionRate'])
            self.male_adv   = self._interpolate(self.data['AdvancedProgressionRate'])
            # LifeTable: 0번 컬럼=남성, 1번 컬럼=여성
            self.male_life  = self.data['LifeTable'][:, 0]

            # 여성 데이터 (변수명이 존재하면 로드, 없으면 남성 데이터 사용)
            if 'new_polyp_female' in self.data.dtype.names:
                self.female_onset = self._interpolate(self.data['new_polyp_female'])
                self.female_early = self._interpolate(self.data['early_progression_female'])
                self.female_adv   = self._interpolate(self.data['advanced_progression_female'])
                self.female_life  = self.data['LifeTable'][:, 1]
            else:
                self.female_onset, self.female_early, self.female_adv = self.male_onset, self.male_early, self.male_adv
                self.female_life = self.male_life

            # [공통 생물학]
            # Healing: 용종이 저절로 사라질 확률 (크기별)
            self.prob_healing = self.data['Healing'].flatten() 
            # FastCancer: 용종 단계를 건너뛰고 빠르게 암이 될 확률
            self.prob_fast_cancer = self.data['FastCancer'].flatten()
            # Symptoms: 암 기수별 증상 발현율
            self.prob_symptoms = self.data['Symptoms'].flatten()
            # IndividualRisk: 사람마다 다른 위험도 분포
            self.risk_dist = self.data['IndividualRisk'].flatten()

            # =========================================================
            # [3] 검사 성능 및 합병증 (Screening & Complications)
            # =========================================================
            # 내시경 민감도 (용종 크기별 발견 확률)
            self.sens_colo = self.data['Colo_Detection'].flatten()
            
            # 내시경 합병증 확률 (천공, 출혈) -> AI가 무분별한 검사를 못하게 막는 제동장치
            self.risk_perf  = self._get_scalar('Colonoscopy_RiscPerforation')
            self.risk_bleed = self._get_scalar('Colonoscopy_RiscBleeding')
            self.death_perf = self._get_scalar('DeathPerforation') # 천공 시 사망률

        else:
            # 로드 실패 시 기본 더미 값 (오류 방지용)
            self._set_dummy_values()

        # [4] 고정값 (Benchmark에서 가져온 FOBT 성능)
        self.sens_fobt = np.array([0.02, 0.02, 0.05, 0.05, 0.12, 0.12, 0.4, 0.4, 0.4, 0.4])

        # [5] 강화학습 보상 체계 (Reward Function)
        self.rewards = {
            'cost_wait': 0,
            'cost_fobt': 0.1,
            'cost_colo': 0.5,           # 내시경 비용
            'reward_alive': 0.1,        # 1년 생존 보상
            'reward_polyp': 5.0,        # 용종 제거 (예방)
            'reward_cancer': 100.0,     # 암 발견 (치료)
            'penalty_death': -5000.0,   # 사망 페널티
            'penalty_complication': -50.0 # 합병증(천공/출혈) 발생 시 페널티
        }

    # --- 내부 헬퍼 함수들 ---
    def _get_scalar(self, key):
        """ 변수가 있으면 값(float)을 꺼내고, 없으면 0.0 반환 """
        if key in self.data.dtype.names:
            val = self.data[key]
            return val.item() if val.size == 1 else val.flatten()[0]
        return 0.0

    def _interpolate(self, raw_data, target_len=100):
        """ (1, 20) 데이터를 100세까지 선형 보간 """
        if raw_data is None or raw_data.size == 0: return np.zeros(target_len)
        y = raw_data.flatten()
        x = np.linspace(0, target_len, len(y))
        return np.interp(np.arange(target_len), x, y)
    
    def _set_dummy_values(self):
        """ 파일 없을 때를 위한 기본값 """
        self.male_onset = np.linspace(0.01, 0.05, 100)
        self.male_early = np.full(100, 0.1)
        self.male_adv = np.full(100, 0.1)
        self.male_life = np.linspace(0, 1, 100)
        self.female_onset = self.male_onset
        self.prob_healing = np.zeros(10)
        self.risk_dist = np.ones(100)
        self.sens_colo = np.full(10, 0.9)
        self.risk_perf = 0.001

    # --- 외부 호출용 인터페이스 ---
    def get_rates(self, is_female):
        """ 성별에 맞는 확률 세트(발생, 성장, 암화, 사망) 반환 """
        if is_female:
            return self.female_onset, self.female_early, self.female_adv, self.female_life
        return self.male_onset, self.male_early, self.male_adv, self.male_life
    
    def get_sensitivity(self, test_type, state_idx):
        """ 검사 종류와 병변 상태(0~9)에 따른 민감도 반환 """
        if state_idx < 0: return 0
        idx = min(state_idx, 9) # 인덱스 초과 방지
        
        if test_type == 'colonoscopy':
            return self.sens_colo[idx]
        elif test_type == 'fobt':
            return self.sens_fobt[idx]
        return 0

In [ ]:
#env
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class CmostEnv(gym.Env):
    def __init__(self):
        super(CmostEnv, self).__init__()
        self.cfg = CMOSTConfig() # 앞서 만든 Config (Risk, Gender 포함된 버전)
        
        # [Action Space] 0:Wait, 1:FOBT, 2:Colonoscopy
        self.action_space = spaces.Discrete(3)
        
        # [Observation Space]
        # 0: 나이, 1: 성별, 2: 증상, 3: 마지막검사, 4: 결과, 5: 경과시간
        self.observation_space = spaces.Box(low=0, high=1, shape=(6,), dtype=np.float32)

    # def reset(self, seed=None, options=None):
    #     super().reset(seed=seed)
        
    #     # -------------------------------------------------------
    #     # 1. 환자 생성 (Initialization)
    #     # -------------------------------------------------------
    #     self.max_age = 100 # MATLAB: y < 100
        
    #     # 성별 및 개인 위험도 결정 (MATLAB Initialize 파트)
    #     self.is_female = np.random.rand() < self.cfg.frac_female
    #     self.risk_factor = np.random.choice(self.cfg.risk_dist)
        
    #     # 성별에 따른 확률표 세팅
    #     self.rates = self.cfg.get_rates(self.is_female) 
    #     # (onset, early, adv, life_table) 튜플 언패킹
    #     self.r_onset, self.r_early, self.r_adv, self.r_life = self.rates

    #     # 상태 변수 초기화
    #     self.hidden_polyps = []   # 용종 크기 리스트 (1~9단계)
    #     self.hidden_cancer = None # 암 기수 (None, 1, 2, 3, 4)
        
    #     # -------------------------------------------------------
    #     # 2. Warm-up Simulation (0세 -> 50세)
    #     # MATLAB은 처음부터 돌리므로, 우리도 50세 상태를 만들어야 함
    #     # -------------------------------------------------------
    #     self.age = 0
    #     target_start_age = 50
        
    #     # 50세가 될 때까지 치료 없이 자연사(Biology)만 진행
    #     for _ in range(target_start_age):
    #         # 매년 4분기(Quarterly) 업데이트
    #         for q in range(4):
    #             self._update_biology_quarterly(self.age + q/4.0)
    #         self.age += 1
            
    #     # 50세 시점의 상태 설정
    #     self.last_test_type = 0
    #     self.last_test_result = 0
    #     self.time_since_test = 10.0 # 10년 이상 (검사 안 함)
    #     self.has_symptom = 0
        
    #     # 만약 Warm-up 도중 암으로 사망했거나 진행됐다면 다시 리셋 (RL 학습 효율 위해)
    #     if self.hidden_cancer and self.hidden_cancer >= 4:
    #         return self.reset() # 너무 아픈 환자는 스킵하고 다시 뽑기

    #     return self._get_obs(), {}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # [수정] 무한 재귀 방지를 위한 루프 방식 도입
        max_retries = 100 # 최대 100번까지만 다시 시도
        attempt = 0
        
        while attempt < max_retries:
            attempt += 1
            
            # -------------------------------------------------------
            # 1. 환자 생성 (Initialization)
            # -------------------------------------------------------
            self.max_age = 100
            
            # 성별 및 개인 위험도 결정
            self.is_female = np.random.rand() < self.cfg.frac_female
            self.risk_factor = np.random.choice(self.cfg.risk_dist)
            
            # 성별에 따른 확률표 세팅
            self.rates = self.cfg.get_rates(self.is_female) 
            self.r_onset, self.r_early, self.r_adv, self.r_life = self.rates

            # 상태 변수 초기화
            self.hidden_polyps = []   
            self.hidden_cancer = None 
            
            # -------------------------------------------------------
            # 2. Warm-up Simulation (0세 -> 50세)
            # -------------------------------------------------------
            self.age = 0
            target_start_age = 50
            is_dead_during_warmup = False
            
            # 50세가 될 때까지 자연사 진행
            for _ in range(target_start_age):
                for q in range(4):
                    current_time = self.age + q/4.0
                    self._update_biology_quarterly(current_time)
                    
                    # 만약 Warm-up 도중 자연사했거나 암으로 너무 악화되면 실패 처리
                    if self._check_death(current_time): # 자연사 체크
                        is_dead_during_warmup = True
                        break
                        
                if is_dead_during_warmup: break
                self.age += 1
                
            # [조건 체크] 
            # 1. Warm-up 중 죽지 않았어야 함
            # 2. 50세 시점에 이미 말기 암(4기)이면 안 됨 (RL 학습용 환자로 부적합)
            if not is_dead_during_warmup:
                if not (self.hidden_cancer and self.hidden_cancer >= 4):
                    break # 성공! 루프 탈출
        
        # -------------------------------------------------------
        # [안전장치] 100번 시도해도 실패하면? -> 강제로 초기화 (Reset Force)
        # -------------------------------------------------------
        if attempt >= max_retries:
            # "운 나쁜 환자들은 다 걸러내고, 그냥 평범한 환자로 강제 시작"
            self.age = 50
            self.hidden_polyps = []
            self.hidden_cancer = None # 암 없음으로 강제 설정
        
        # 3. 50세 시점의 관측 정보 설정
        self.last_test_type = 0
        self.last_test_result = 0
        self.time_since_test = 10.0 
        self.has_symptom = 0
        
        # 증상 업데이트 (현재 암 상태에 맞춰서)
        if self.hidden_cancer:
             symp_prob = self.cfg.prob_symptoms[min(5+self.hidden_cancer, 9)]
             if np.random.rand() < symp_prob:
                 self.has_symptom = 1

        return self._get_obs(), {}
    

    def step(self, action):
        reward = 0
        done = False
        info = {'cause': 'alive', 'action': action}
        
        # =========================================================
        # [Time Loop] 1 Step = 1 Year (4 Quarters)
        # MATLAB: while loop 내부
        # =========================================================
        
        # 1. Action 수행 (연초에 결정한다고 가정)
        if not done:
            reward_act, done_act, info_act = self._perform_action(action)
            reward += reward_act
            if done_act: 
                done = True
                info.update(info_act)

        # 2. 분기별 생물학적 변화 (Quarterly Loop)
        # MATLAB: for q = 1:4
        if not done:
            for q in range(4):
                current_time = self.age + q/4.0
                
                # A. 자연사 (Biology Update)
                self._update_biology_quarterly(current_time)
                
                # B. 증상 발현 (Symptom Development)
                # MATLAB: if time >= Ca.SympTime
                if self.hidden_cancer:
                    symp_prob = self.cfg.prob_symptoms[min(5+self.hidden_cancer, 9)] / 4.0 # 분기 보정
                    if np.random.rand() < symp_prob:
                        self.has_symptom = 1
                        # 증상이 심하면 병원 방문 -> 암 발견 및 치료 시도 (간소화)
                        # 여기서는 RL 에이전트가 증상을 보고 다음 Step에 Action을 취하도록 유도
                
                # C. 사망 체크 (Natural & Cancer)
                # MATLAB: people die of natural causes / cancer
                if self._check_death(current_time):
                    done = True
                    # 암 사망인지 자연사인지 판별
                    if self.hidden_cancer and self.has_symptom:
                        reward += self.cfg.rewards['penalty_death']
                        info['cause'] = 'cancer_death'
                    else:
                        info['cause'] = 'natural_death' # 자연사는 페널티 없음
                    break

        # 3. 년도 마무리
        if not done:
            self.age += 1
            reward += self.cfg.rewards['reward_alive'] # 생존 보상
            self.time_since_test += 1
            
            if self.age >= self.max_age:
                done = True
                info['cause'] = 'max_age_reached'

        return self._get_obs(), reward, done, False, info

    # -------------------------------------------------------------
    # Helper Methods (MATLAB Logic Implementation)
    # -------------------------------------------------------------

    def _perform_action(self, action):
        """ 검사 수행 및 결과 처리 """
        r, d = 0, False
        i = {}
        
        if action == 0: # Wait
            return r, d, i
            
        elif action == 1: # FOBT
            r -= self.cfg.rewards['cost_fobt']
            self.time_since_test = 0
            self.last_test_type = 0.5
            
            detected, f_type = self._run_test('fobt', remove=False)
            if detected:
                self.last_test_result = f_type
                r += 0.5 # 발견 보상
            else:
                self.last_test_result = 0
                
        elif action == 2: # Colonoscopy
            r -= self.cfg.rewards['cost_colo']
            self.time_since_test = 0
            self.last_test_type = 1.0
            
            # 합병증 체크 (MATLAB: Complications if rand < risc...)
            if np.random.rand() < self.cfg.risk_perf:
                r += self.cfg.rewards['penalty_complication']
                if np.random.rand() < self.cfg.death_perf:
                    return r + self.cfg.rewards['penalty_death'], True, {'cause': 'perforation_death'}

            # 검사 수행
            detected, f_type = self._run_test('colonoscopy', remove=True)
            if detected:
                self.last_test_result = f_type
                if f_type == 1: r += self.cfg.rewards['reward_polyp']
                elif f_type == 2:
                    r += self.cfg.rewards['reward_cancer']
                    d = True # 치료 성공 종료
                    i['cause'] = 'cancer_detected'
            else:
                self.last_test_result = 0
                
        return r, d, i

    def _run_test(self, t_type, remove):
        """ 민감도 적용 검사 """
        det, f_type = False, 0
        
        # 암 탐지
        if self.hidden_cancer:
            sens = self.cfg.get_sensitivity(t_type, 5 + self.hidden_cancer)
            if np.random.rand() < sens: return True, 2
            
        # 용종 탐지
        rem_idx = []
        for idx, size in enumerate(self.hidden_polyps):
            sens = self.cfg.get_sensitivity(t_type, min(size-1, 5))
            if np.random.rand() < sens:
                det = True
                f_type = 1
                rem_idx.append(idx)
        
        # 내시경이면 제거 (MATLAB: we delete the current polyp)
        if remove and rem_idx:
            for idx in sorted(rem_idx, reverse=True):
                self.hidden_polyps.pop(idx)
                
        return det, f_type

    def _update_biology_quarterly(self, current_time):
        """ 
        MATLAB NumberCrunching의 핵심 로직 
        (분기별로 호출되므로 확률을 1/4로 조정하여 적용)
        """
        age_idx = min(int(self.age), 99)
        dt = 0.25 # 1분기
        
        # 1. New Polyp (용종 발생)
        # MATLAB: PolypRate = IndividualRisk * NewPolyp(y) * gender_factor
        # 분기별 확률 근사: 1 - (1-p)^(1/4) ~ p/4
        prob_onset = (self.r_onset[age_idx] * self.risk_factor) * dt
        if np.random.rand() < prob_onset:
            self.hidden_polyps.append(1) # 크기 1 생성

        # 2. Fast Cancer (Direct Cancer)
        # MATLAB: rand < DirectCancerRate * DirectCancerSpeed
        prob_fast = self.cfg.prob_fast_cancer[age_idx] if age_idx < len(self.cfg.prob_fast_cancer) else 0
        if np.random.rand() < prob_fast * dt:
            # 바로 암 발생 (용종 단계 건너뜀)
            self.hidden_cancer = 1 # 1기 암
            
        # 3. Polyp Progression (용종 성장)
        # MATLAB: AgeProgression * Location * Gender * Risk
        for i in range(len(self.hidden_polyps)):
            size = self.hidden_polyps[i]
            if size <= 3:
                rate = self.r_early[age_idx]
            else:
                rate = self.r_adv[age_idx]
            
            if np.random.rand() < rate * dt: # 분기별 성장 확률
                self.hidden_polyps[i] += 1

        # 4. Healing (자연 치유/소멸)
        # MATLAB: a polyp shrinks or disappears
        for i in reversed(range(len(self.hidden_polyps))):
            size = self.hidden_polyps[i]
            # Healing 데이터가 10개(인덱스 0~9)라고 가정
            heal_prob = self.cfg.prob_healing[min(size-1, 9)] if size > 0 else 0
            if np.random.rand() < heal_prob * dt:
                self.hidden_polyps[i] -= 1 # 크기 감소
                if self.hidden_polyps[i] <= 0:
                    self.hidden_polyps.pop(i) # 소멸

        # 5. Malignancy (암화)
        # 용종이 너무 커지면 암으로 변함
        for i in reversed(range(len(self.hidden_polyps))):
            if self.hidden_polyps[i] > 6: # 6단계 초과 시
                # MATLAB에서는 바로 7(암)로 변환
                if self.hidden_cancer is None:
                    self.hidden_cancer = 1
                self.hidden_polyps.pop(i)

        # 6. Cancer Progression (암 진행)
        if self.hidden_cancer and self.hidden_cancer < 4:
            # 암은 빨리 자람 (임의 확률 0.1 * dt)
            if np.random.rand() < 0.4 * dt: 
                self.hidden_cancer += 1

    def _check_death(self, current_time):
        """ 자연사 및 암 사망 체크 """
        age_idx = min(int(self.age), 99)
        
        # 1. 자연사 (LifeTable)
        # MATLAB: if rand < (LifeTable(y)/4)
        nat_death_prob = self.r_life[age_idx] / 4.0
        if np.random.rand() < nat_death_prob:
            return True
            
        # 2. 암 사망 (Cancer Death)
        if self.hidden_cancer:
            # 암이 심할수록 사망 확률 증가 (4기면 매우 높음)
            cancer_death_base = [0, 0.01, 0.05, 0.2, 0.8] # 0~4기 사망률(연간) 예시
            c_death = cancer_death_base[self.hidden_cancer] / 4.0
            if np.random.rand() < c_death:
                return True
                
        return False

    def _get_obs(self):
        return np.array([
            (self.age - 50) / 50.0,      # 나이 (50~100 정규화)
            1.0 if self.is_female else 0.0,
            self.has_symptom,
            self.last_test_type,
            self.last_test_result,
            min(self.time_since_test, 10) / 10.0
        ], dtype=np.float32)

In [ ]:
#ppo 학습. 
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 1. 환경 설정 및 모델 생성
# ==========================================

# DummyVecEnv: SB3에서 학습 속도를 높이기 위해 환경을 감싸는 래퍼(Wrapper)
# (여러 환경을 동시에 돌릴 수도 있지만 여기선 1개만 사용)
env = DummyVecEnv([lambda: CmostEnv()])

print("🤖 AI 모델 생성 중...")

# PPO 모델 정의
model = PPO(
    "MlpPolicy",         # 입력이 이미지(CNN)가 아니라 수치(Vector)이므로 MLP 사용
    env, 
    verbose=1,           # 학습 로그 출력 (1: 정보 출력)
    learning_rate=0.0003, # 학습 속도 (너무 빠르면 발산, 느리면 답답함)
    gamma=0.999,         # [핵심] 할인율 (0.99 -> 0.999)
                         # 미래의 보상(사망 방지)을 현재의 비용(검사비)보다 중요하게 여김
    ent_coef=0.01,       # 탐험 계수 (다양한 시도를 해보도록 유도)
    batch_size=64,       # 한 번에 학습할 데이터 양
    seed=42              # 재현성을 위한 시드 고정
)

# ==========================================
# 2. 학습 전 성능 테스트 (Baseline)
# ==========================================
def evaluate_agent(agent_model, num_episodes=20, name="Agent"):
    """ 에이전트 성능을 테스트하는 함수 """
    test_env = CmostEnv() # 테스트용 환경 별도 생성
    rewards = []
    lifespans = []
    deaths_cancer = 0
    deaths_perf = 0
    check_counts = {'fobt': 0, 'colo': 0}

    print(f"\n--- [{name}] 평가 시작 ({num_episodes}명) ---")
    
    for ep in range(num_episodes):
        obs, _ = test_env.reset()
        done = False
        ep_reward = 0
        
        while not done:
            if agent_model:
                action, _ = agent_model.predict(obs, deterministic=True)
            else:
                action = test_env.action_space.sample() # 랜덤 행동
            
            obs, reward, done, _, info = test_env.step(action)
            ep_reward += reward
            
            # 통계 집계
            if info.get('action') == 1: check_counts['fobt'] += 1
            if info.get('action') == 2: check_counts['colo'] += 1
            
        rewards.append(ep_reward)
        lifespans.append(test_env.age)
        
        if info.get('cause') == 'cancer_death': deaths_cancer += 1
        if info.get('cause') == 'perforation_death': deaths_perf += 1

    avg_reward = np.mean(rewards)
    avg_life = np.mean(lifespans)
    
    print(f"평균 보상: {avg_reward:.2f}")
    print(f"평균 수명: {avg_life:.1f}세")
    print(f"암 사망률: {(deaths_cancer/num_episodes)*100:.1f}%")
    print(f"합병증 사망: {(deaths_perf/num_episodes)*100:.1f}%")
    print(f"총 검사량: 내시경 {check_counts['colo']}회, FOBT {check_counts['fobt']}회")
    return avg_reward

print("\n📊 학습 전 (Random) 성능 측정...")
evaluate_agent(None, num_episodes=50, name="Random Agent")

# ==========================================
# 3. 모델 학습 (Training)
# ==========================================
print("\n🔥 학습 시작! (약 3~5분 소요 예상)")
# total_timesteps: 시뮬레이션 총 횟수 (년 단위)
# 20만~30만 번 정도면 간단한 정책은 수립됩니다.
model.learn(total_timesteps=300000) 
print("✅ 학습 완료!")

# 모델 저장
model.save("cmost_ppo_model")
print("💾 모델 저장 완료: cmost_ppo_model.zip")

# ==========================================
# 4. 학습 후 성능 테스트 (Evaluation)
# ==========================================
print("\n📊 학습 후 (PPO AI) 성능 측정...")
evaluate_agent(model, num_episodes=50, name="Trained AI")

# ==========================================
# 5. 시각화 (한 명의 환자 인생 따라가기)
# ==========================================
print("\n🔍 [상세 분석] 학습된 AI의 진료 기록 (1명 예시)")
test_env = CmostEnv()
obs, _ = test_env.reset()
done = False
print(f"환자 시작: {test_env.age}세 (여성={test_env.is_female}, 위험도={test_env.risk_factor:.2f})")

history = []
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = test_env.step(action)
    
    act_str = ["대기", "FOBT", "내시경"][action]
    if action > 0 or done: # 검사했거나 죽었을 때만 로그 출력
        print(f"[{test_env.age}세] 행동: {act_str} | 보상: {reward:.1f} | 상태: {info['cause']}")

🤖 AI 모델 생성 중...
Using cuda device

📊 학습 전 (Random) 성능 측정...

--- [Random Agent] 평가 시작 (50명) ---
평균 보상: -8.61
평균 수명: 90.0세
암 사망률: 0.0%
합병증 사망: 0.0%
총 검사량: 내시경 0회, FOBT 0회

🔥 학습 시작! (약 3~5분 소요 예상)


c:\Users\daybr\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 1295 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1047       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.02567853 |
|    clip_fraction        | 0.275      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.08      |
|    explained_variance   | 0.025      |
|    learning_rate        | 0.0003     |
|    loss                 | 1.1        |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0328    |
|    value_loss           | 3.48       |
----------------------------------------
-----------------------------------------
| time/   

KeyboardInterrupt: 

In [ ]:
#evaluation (vs random)

# --- [5] 학습된 모델 성능 테스트 ---

def run_simulation(agent_model, episodes=10):
    """ 에이전트를 이용해 환자 시뮬레이션을 돌리고 결과를 출력하는 함수 """
    # 테스트용 환경 별도 생성
    test_env = CmostEnv()
    total_rewards = []
    lifespans = []
    deaths_by_cancer = 0
    
    print(f"\n--- 시뮬레이션 시작 ({episodes}명 환자) ---")
    
    for ep in range(episodes):
        obs, _ = test_env.reset()
        done = False
        ep_reward = 0
        actions = []
        
        while not done:
            # 모델이 있으면 예측값 사용, 없으면 랜덤 행동
            if agent_model:
                action, _ = agent_model.predict(obs, deterministic=True)
            else:
                action = test_env.action_space.sample()
                
            obs, reward, done, _, info = test_env.step(action)
            ep_reward += reward
            actions.append(action)
            
        total_rewards.append(ep_reward)
        lifespans.append(test_env.age)
        if info.get('cause_of_death') == 'cancer':
            deaths_by_cancer += 1
            
        # 처음 3명만 로그 자세히 출력
        if ep < 3:
            print(f"환자 {ep+1}: 사망나이 {test_env.age}세, 사인: {info.get('cause_of_death')}")
            # 행동 통계 (0:대기, 1:FOBT, 2:내시경)
            unique, counts = np.unique(actions, return_counts=True)
            action_dict = dict(zip(unique, counts))
            print(f"   -> 검사 횟수: 내시경 {action_dict.get(2, 0)}회, FOBT {action_dict.get(1, 0)}회")
            print(f"   -> 총 보상: {ep_reward:.2f}")

    print("-" * 30)
    print(f"평균 수명: {np.mean(lifespans):.1f}세")
    print(f"평균 보상: {np.mean(total_rewards):.2f}")
    print(f"암 사망률: {(deaths_by_cancer/episodes)*100:.1f}%")

# 1. 학습 전 (랜덤) 성능
print("\n[비교 1] 학습 전 (Random Agent)")
run_simulation(None, episodes=100)

# 2. 학습 후 (PPO) 성능
print("\n[비교 2] 학습 후 (PPO Agent)")
run_simulation(model, episodes=100)


[비교 1] 학습 전 (Random Agent)
Loading Optimized Configuration from /Users/llnnseoii/Desktop/cmost/Settings/CMOST13.mat...
✅ 데이터 로드 성공: 핵심 변수 추출 및 보간 시작

--- 시뮬레이션 시작 (100명 환자) ---
환자 1: 사망나이 55세, 사인: None
   -> 검사 횟수: 내시경 1회, FOBT 3회
   -> 총 보상: -0.30
환자 2: 사망나이 58세, 사인: None
   -> 검사 횟수: 내시경 2회, FOBT 3회
   -> 총 보상: 99.50
환자 3: 사망나이 94세, 사인: None
   -> 검사 횟수: 내시경 16회, FOBT 15회
   -> 총 보상: 124.90
------------------------------
평균 수명: 71.4세
평균 보상: 30.52
암 사망률: 0.0%

[비교 2] 학습 후 (PPO Agent)
Loading Optimized Configuration from /Users/llnnseoii/Desktop/cmost/Settings/CMOST13.mat...
✅ 데이터 로드 성공: 핵심 변수 추출 및 보간 시작

--- 시뮬레이션 시작 (100명 환자) ---
환자 1: 사망나이 72세, 사인: None
   -> 검사 횟수: 내시경 20회, FOBT 3회
   -> 총 보상: 92.40
환자 2: 사망나이 87세, 사인: None
   -> 검사 횟수: 내시경 0회, FOBT 0회
   -> 총 보상: 3.70
환자 3: 사망나이 79세, 사인: None
   -> 검사 횟수: 내시경 2회, FOBT 28회
   -> 총 보상: 100.10
------------------------------
평균 수명: 72.4세
평균 보상: 32.41
암 사망률: 0.0%


In [ ]:
import pandas as pd
import numpy as np

# 판다스 출력 제한 해제 (중요: 이 설정을 해야 중간 생략이 안 됩니다)
pd.set_option('display.max_rows', None)      # 행 개수 제한 해제
pd.set_option('display.max_columns', None)   # 열 개수 제한 해제
pd.set_option('display.width', 1000)         # 가로 길이 늘리기
pd.set_option('display.max_colwidth', None)  # 셀 내용 잘림 방지

def get_full_patient_list(model, env_class, num_patients=100):
    env = env_class()
    records = []
    
    print(f"🔍 {num_patients}명의 환자 기록 생성 중...", end="")
    
    for i in range(num_patients):
        if i % 10 == 0: print(".", end="") # 진행상황 표시
        
        obs, _ = env.reset()
        done = False
        
        # 기록용 변수
        risk = env.risk_factor
        events = [] # (나이, 검사종류)
        cause_of_death = None
        
        while not done:
            # AI 행동 결정
            if model:
                action, _ = model.predict(obs, deterministic=True)
            else:
                action = env.action_space.sample()
            
            current_age = env.age
            
            # 검사 기록 (Action 1: FOBT, 2: Colo)
            if action == 1:
                events.append((current_age, "FOBT"))
            elif action == 2:
                events.append((current_age, "Colo"))
            
            obs, reward, done, _, info = env.step(action)
            
            if done:
                cause_of_death = info.get('cause')

        # --- 데이터 가공 ---
        # 1. 검사 종류 (Methods)
        methods = [e[1] for e in events]
        
        # 2. 검사 주기 (Intervals)
        test_ages = [e[0] for e in events]
        if len(test_ages) > 1:
            # 나이 차이 계산 (예: 50세, 55세 -> 5년 간격)
            intervals = np.diff(test_ages).tolist() 
            # 보기 좋게 소수점 제거 (5.0 -> 5)
            intervals = [int(x) for x in intervals]
        else:
            intervals = "-" # 검사가 0회 또는 1회라 간격 없음

        # 3. 사망 원인 한글화 (보기 좋게)
        cause_map = {
            "natural_death": "자연사",
            "cancer_death": "암 사망",
            "perforation_death": "의료사고(천공)",
            "max_age_reached": "수명종료(100세)",
            "cancer_detected": "암 완치(발견)"
        }
        cause_str = cause_map.get(cause_of_death, cause_of_death)

        # 리스트에 추가
        records.append({
            "ID": i + 1,
            "Death Cause": cause_str,
            "Risk": round(risk, 2),
            "Total Tests": len(events),
            "Methods": str(methods).replace("'", ""), # 따옴표 제거해서 깔끔하게
            "Intervals (Years)": str(intervals)
        })

    print(" 완료!\n")
    return pd.DataFrame(records)

# --- 실행 및 출력 ---
# 1. 데이터 생성
df_full_list = get_full_patient_list(model, CmostEnv, num_patients=100)

# 2. 전체 출력 (to_string()을 쓰면 잘리지 않고 문자열로 다 나옵니다)
print(df_full_list.to_string(index=False))

Loading Optimized Configuration from /Users/llnnseoii/Desktop/cmost/Settings/CMOST13.mat...
✅ 데이터 로드 성공: 핵심 변수 추출 및 보간 시작
🔍 100명의 환자 기록 생성 중............. 완료!

 ID Death Cause  Risk  Total Tests                                                                                                                                                                                                                                                                                          Methods                                                                                                                             Intervals (Years)
  1         자연사 48.07            0                                                                                                                                                                                                                                                                                               []                                                    

In [ ]:
df_full_list.to_csv('AI_Doctor_Report.csv', index=False)


✅ 저장 완료! 'AI_Doctor_Report.csv' 파일을 열어보세요.
저장 경로: /Users/llnnseoii/Desktop/cmost/AI_Doctor_Report.csv


In [ ]:
df=pd.read_csv('AI_Doctor_Report.csv')
df

,ID,Death Cause,Risk,Total Tests,Methods,Intervals (Years)
0,1,자연사,48.07,0,NaN,-
1,2,자연사,0.58,0,NaN,-
2,3,자연사,3.65,0,NaN,-
3,4,암 완치(발견),1.53,10,"FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, Colo","1, 1, 1, 1, 1, 1, 1, 1, 1"
4,5,자연사,3.45,0,NaN,-
5,6,암 완치(발견),54.05,5,"FOBT, FOBT, FOBT, Colo, Colo","1, 1, 1, 1"
6,7,자연사,3.18,0,NaN,-
7,8,자연사,0.03,0,NaN,-
8,9,자연사,2.36,0,NaN,-
9,10,자연사,0.42,11,"FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, FOBT, FOBT","1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
